# Pipeline using application/ld+json works for
- nos.nl (getting search results is possible and loopable)
- cnn (getting search results is possible and loopable)
- fox news (verified freely accessible)
- cbs news (but getting search results is hard)
- nbc news

# Pipeline using application/ld+json does not work for
- abc news, which should be accessed via the "body" element of another script (which unfortunately has no name)
- 

In [10]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import requests
import time

In [11]:
#Load page
driver = webdriver.Chrome()
driver.get("https://www.foxnews.com/search-results/search?q=AI")
time.sleep(5)

In [ ]:
#Close the banner in front of load-more
element = driver.find_element(By.CSS_SELECTOR, ".close")
element.click()

In [8]:
#Code to click the load-more button on Fox News search results page
for i in range(30):
    element = driver.find_element(By.CSS_SELECTOR, ".button.load-more")
    element.click()
    time.sleep(1)

In [9]:
# Gets all links on a page using selenium
elements = driver.find_elements(By.TAG_NAME, "a")
for element in elements:
    print(element.get_dom_attribute("href"))

https://www.foxnews.com/
https://www.foxbusiness.com/
https://nation.foxnews.com/
https://radio.foxnews.com/
https://www.foxweather.com/
https://www.outkick.com/
https://www.foxnews.com/books
https://www.foxnews.com/
/us
/politics
/world
/opinion
/media
/entertainment
/sports
/lifestyle
/video
/category/tech/artificial-intelligence
#
#
#
https://www.foxnews.com/video/5614615980001
#
https://www.foxnews.com/politics/ny-v-trump-jury-verdict-former-presidents-historic-criminal-trial
https://www.foxnews.com/politics/trump-vows-fight-end-promising-this-long-from-over-after-guilty-verdict
https://www.foxnews.com/politics/trump-ny-sentencing-days-before-republican-convention
https://www.foxnews.com/live-news/may-30-trump-trial-jury-deliberations-day-2
//www.foxnews.com/us
//www.foxnews.com/category/us/crime
//www.foxnews.com/category/us/military
//www.foxnews.com/category/us/education
//www.foxnews.com/category/us/terror
//www.foxnews.com/category/us/immigration
//www.foxnews.com/category/us/

In [1]:
from bs4 import BeautifulSoup
from html import parser
import re

In [2]:
#Gets all search hit pages from searching for AI on NOS.nl
d = {}
for i in range(30)[1:]:
    with open(f"pages/nos-ai-{i}.html") as fp:
        soup = BeautifulSoup(fp, 'html.parser')
        d[i] = soup

In [3]:
#Gets all relevant_links from the search results for AI on NOS.nl
relevant_links = []

for i in d.keys():
    record = False
    zoeken_counter = 0
    for link in d[i].find_all('a'):
        if link.get('href')[:8] == "/zoeken?":
            record = False
        if record and re.search(r"artikel",link.get("href")):
            relevant_links.append("https://www.nos.nl" + link.get('href'))
        if link.get('href') == "/zoeken":
            zoeken_counter+=1
        if zoeken_counter == 2:
            record = True
            zoeken_counter = 0

In [4]:
help(BeautifulSoup)

Help on class BeautifulSoup in module bs4:

class BeautifulSoup(bs4.element.Tag)
 |  BeautifulSoup(markup='', features=None, builder=None, parse_only=None, from_encoding=None, exclude_encodings=None, element_classes=None, **kwargs)
 |  
 |  A data structure representing a parsed HTML or XML document.
 |  
 |  Most of the methods you'll call on a BeautifulSoup object are inherited from
 |  PageElement or Tag.
 |  
 |  Internally, this class defines the basic interface called by the
 |  tree builders when converting an HTML/XML document into a data
 |  structure. The interface abstracts away the differences between
 |  parsers. To write a new tree builder, you'll need to understand
 |  these methods as a whole.
 |  
 |  These methods will be called by the BeautifulSoup constructor:
 |    * reset()
 |    * feed(markup)
 |  
 |  The tree builder may call these methods from its feed() implementation:
 |    * handle_starttag(name, attrs) # See note about return value
 |    * handle_endtag(na

In [5]:
#Gets the soup for all files in relevant_links[:10]
import os
soups = []
for link in relevant_links[:10]:
    os.system(f"wget -O link.html {link}")
    with open("link.html","r") as file:
        soups.append(BeautifulSoup(file, 'html.parser'))

--2024-06-04 13:26:58--  https://www.nos.nl/artikel/2517922-zorgen-bij-muzikanten-over-muziek-producerende-ai-tool
Resolving www.nos.nl (www.nos.nl)... 18.239.18.12, 18.239.18.80, 18.239.18.74, ...
Connecting to www.nos.nl (www.nos.nl)|18.239.18.12|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://nos.nl/artikel/2517922-zorgen-bij-muzikanten-over-muziek-producerende-ai-tool [following]
--2024-06-04 13:26:58--  https://nos.nl/artikel/2517922-zorgen-bij-muzikanten-over-muziek-producerende-ai-tool
Resolving nos.nl (nos.nl)... 18.239.50.117, 18.239.50.44, 18.239.50.102, ...
Connecting to nos.nl (nos.nl)|18.239.50.117|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 144775 (141K) [text/html]
Saving to: ‘link.html’

     0K .......... .......... .......... .......... .......... 35% 1,85M 0s
    50K .......... .......... .......... .......... .......... 70% 2,81M 0s
   100K .......... .......... .......... .......... . 

In [28]:
#Get the json-file containing file content and metadata for all soup in soups
#and turn into dictionary (this has been tested for NOS.nl and for one article on CNN.com; 'application/ld+json' seems to be a standard
#type for scripts describing the metadata and content of news site articles)
import json
import requests

url = "https://www.cnn.com/2024/04/29/tech/south-korea-samsung-profits-ai-boom-hnk-intl/"
req = requests.get(url)
soup = BeautifulSoup(req.content,'html.parser')
contents = []
for script in soup.find_all("script"):
    try:
        if script['type'] == 'application/ld+json':
            contents.append(json.loads(script.contents[0]))
    except:
        pass
print("contents:",contents)
print(contents[0].keys())
author = contents[0]['author']
headline = contents[0]['headline']
mainText = contents[0]['articleBody']
author, headline, mainText

contents: [{'@type': 'NewsArticle', '@context': 'https://schema.org', 'articleBody': 'Samsung\xa0Electronics\xa0forecast demand for artificial intelligence would hold strong and tighten supply of some high-end chips, joining rivals in benefiting from a solid rebound in the battered global memory chip market. The upbeat outlook from the world’s largest memory chip maker sent its shares 1.8% higher on Tuesday after it reported a more than 10-fold rise in first-quarter operating profit. But so far this year\xa0Samsung\xa0shares are down 0.8%, lagging SK Hynix’s 24% gain, as it seeks to\xa0catch up\xa0with its smaller rival in the supply of top-end chips such as high bandwidth memory (HBM) to AI leader Nvidia (NVDA). “We plan to increase supply of HBM-related chips in 2024 by more than three-fold versus last year,” Jaejune Kim, a\xa0Samsung\xa0vice president in charge of the memory division, said on an earnings call. Samsung\xa0said it began mass production this month of the latest HBM chi

([],
 'Samsung reports enormous jump in profit on AI boom',
 'Samsung\xa0Electronics\xa0forecast demand for artificial intelligence would hold strong and tighten supply of some high-end chips, joining rivals in benefiting from a solid rebound in the battered global memory chip market. The upbeat outlook from the world’s largest memory chip maker sent its shares 1.8% higher on Tuesday after it reported a more than 10-fold rise in first-quarter operating profit. But so far this year\xa0Samsung\xa0shares are down 0.8%, lagging SK Hynix’s 24% gain, as it seeks to\xa0catch up\xa0with its smaller rival in the supply of top-end chips such as high bandwidth memory (HBM) to AI leader Nvidia (NVDA). “We plan to increase supply of HBM-related chips in 2024 by more than three-fold versus last year,” Jaejune Kim, a\xa0Samsung\xa0vice president in charge of the memory division, said on an earnings call. Samsung\xa0said it began mass production this month of the latest HBM chips for use in generative